### Analyse Warnings in Mathlib/Geometry/

In [1]:
PROJECT_NAME = "AnalyseMathlib"
LEAN_VERSION = "v4.15.0"

In [2]:
# Setup a Lean environment in AnalyseMathlib/ manually:

# Automatic is broken due to v4.16.0-rc1 at the time of writing

# lake +leanprover-community/mathlib4:lean-toolchain new AnalyseMathlib math
# cd AnalyseMathlib
# Alter manually: rev = "v4.15.0" in lakefile.toml (mathlib) and set lean-toolchain to same version
# lake update

In [ ]:
# Install leanclient
!pip install leanclient

In [4]:
import leanclient as lc
import os

ANALYSIS_FOLDER = ".lake/packages/mathlib/Mathlib/Geometry"

# Find all .lean files in the folder and its subfolders
lean_files = []
for root, dirs, files in os.walk(f"{PROJECT_NAME}/{ANALYSIS_FOLDER}"):
    root_relative = root[len(f"{PROJECT_NAME}/") :]
    for file in files:
        if file.endswith(".lean"):
            lean_files.append(os.path.join(root_relative, file))

print(f"Found {len(lean_files)} lean files in {ANALYSIS_FOLDER}")

Found 84 lean files in .lake/packages/mathlib/Mathlib/Geometry


In [5]:
# Extract all diagnostics from the lean files
def extract_diagnostics(sfc: lc.SingleFileClient):
    return sfc.get_diagnostics()


pool = lc.LeanClientPool(f"{PROJECT_NAME}", num_workers=4)
with pool:
    diagnostics = pool.map(extract_diagnostics, lean_files, batch_size=4, verbose=True)

print(f"Extracted {sum(len(d) for d in diagnostics)} diagnostic messages")

Processing files: 100%|██████████| 84/84 [02:31<00:00,  1.80s/it]

Extracted 60 diagnostic messages


In [6]:
# Simple statistics
diag_per_file = [f for f in zip(lean_files, diagnostics) if len(f[1]) > 0]
diag_per_file = sorted(diag_per_file, key=lambda x: len(x[1]), reverse=True)

errors = sum(1 for d in diagnostics for diag in d if diag["severity"] == 1)
if errors > 0:
    print(f"Found {errors} errors")

l_folder = len(ANALYSIS_FOLDER) + 1
print("Files with warnings:")
for file, diags in diag_per_file:
    warnings = sum(1 for d in diags if d["severity"] == 2)
    print(f"{file[l_folder:]}: {warnings} warnings")

warnings = [m["message"] for m in sum(diagnostics, []) if m["severity"] == 2]
if len(warnings) == len(set(warnings)):
    print(f"All {len(warnings)} warnings are unique.")

print("\nExample warning:")
print(warnings[-1])

Files with warnings:
Manifold/SmoothManifoldWithCorners.lean: 19 warnings
Manifold/Algebra/Monoid.lean: 4 warnings
Euclidean/Sphere/Basic.lean: 3 warnings
Euclidean/Inversion/Basic.lean: 3 warnings
Manifold/LocalInvariantProperties.lean: 3 warnings
Manifold/MFDeriv/Basic.lean: 3 warnings
Manifold/ContMDiff/Defs.lean: 3 warnings
Manifold/Algebra/SmoothFunctions.lean: 3 warnings
RingedSpace/OpenImmersion.lean: 3 warnings
Manifold/ChartedSpace.lean: 2 warnings
Manifold/MFDeriv/SpecificFunctions.lean: 2 warnings
Euclidean/PerpBisector.lean: 1 warnings
Manifold/PartitionOfUnity.lean: 1 warnings
Manifold/Diffeomorph.lean: 1 warnings
Manifold/ContMDiffMFDeriv.lean: 1 warnings
Manifold/VectorBundle/SmoothSection.lean: 1 warnings
Manifold/MFDeriv/Defs.lean: 1 warnings
Manifold/ContMDiff/Product.lean: 1 warnings
Manifold/ContMDiff/Atlas.lean: 1 warnings
Manifold/Instances/Sphere.lean: 1 warnings
RingedSpace/LocallyRingedSpace.lean: 1 warnings
RingedSpace/SheafedSpace.lean: 1 warnings
RingedSpace